In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent) +'/')
from utils.visualize import img_grid

from torchvision import transforms 

import numpy as np
import pandas as pd
import torchio as tio
import nibabel as nib

import ipywidgets as ipyw
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
import scipy
from sklearn import metrics
import json 

In [2]:
macaw_path = str(Path(os.getcwd()).parent.parent)
ukbb_path = '/home/erik.ohara/UKBB'
ukbb_T1_warped = '/work/forkert_lab/erik/T1_warped/test'
ukbb_T1_warped_folder = '/work/forkert_lab/erik/T1_warped'
generated_path = '/work/forkert_lab/erik/MACAW/cf_images/PCA_five_diff_res_denormalized'
crop_path = '/work/forkert_lab/erik/T1_cropped'
z_initial = 41
z_fim = 140
nsamples = 5
# histogram matching parameters
m_number_histogram_levels = 500
m_number_match_points = 50
m_threshold_mean_intensity = False

In [3]:
data_path = ukbb_path + '/ukbb_img.csv'
df = pd.read_csv(data_path,low_memory=False)

In [4]:
# Getting the generated images
subjects_eid = np.array([])
cf_age = np.array([])
cf_sex = np.array([])
images_generated = []
files_generated = []
for file in tqdm(os.listdir(generated_path)):
    if '.nii' in file:
        files_generated.append(file)
        splits = file.split("_")
        subject_eid = int(splits[0])
        age = splits[1]
        sex = splits[2].split(".")[0]
        subjects_eid = np.append(subjects_eid,subject_eid)
        cf_age = np.append(cf_age, age)
        cf_sex = np.append(cf_sex, sex)
        image_load = nib.load(os.path.join(generated_path, file)).get_fdata()
        images_generated.append(image_load)
print(len(images_generated))
images_generated = np.array(images_generated)

  0%|          | 0/2370 [00:00<?, ?it/s]

2370


In [5]:
# Getting the original images
images_original = []
real_age = []
real_sex = []
for individual in tqdm(subjects_eid):
    individual = int(individual)
    real_age.append(df[df["eid"] == individual]["Age"].item())
    real_sex.append(df[df["eid"] == individual]["Sex"].item())
    image_load = nib.load(os.path.join(ukbb_T1_warped, str(individual) + ".nii.gz")).get_fdata()
    images_original.append(image_load)
real_age = np.array(real_age)
real_sex = np.array(real_sex)
images_original = np.array(images_original)

  0%|          | 0/2370 [00:00<?, ?it/s]

## Cropped

In [6]:
#cropping original image
generated_shape = images_generated[0].shape
original_shape = images_original[0].shape
x_initial = int((original_shape[0] - generated_shape[0])/2)
x_fim = x_initial +  generated_shape[0]
y_initial = int((original_shape[1] - generated_shape[1])/2)
y_fim = y_initial +  generated_shape[1]
images_original = [image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim+1] for image in images_original]

In [7]:
for idx, each_image_nan in tqdm(enumerate(images_original)):
    image_load = nib.load(os.path.join(ukbb_T1_warped, str(int(subjects_eid[idx])) + '.nii.gz'))
    ni_img = nib.Nifti1Image(each_image_nan, image_load.affine, image_load.header)
    nib.save(ni_img, crop_path + "/" + str(int(subjects_eid[idx])) + '.nii.gz')

0it [00:00, ?it/s]

In [8]:
crop_path

'/work/forkert_lab/erik/T1_cropped'

In [9]:
images_original[0].shape

(150, 150, 100)

In [10]:
images_generated[0].shape

(150, 150, 100)